# INSTALL SPARK

In [1]:
# DOWNLOAD DAN INSTALL SPARK, EXTRACT KE CURRENT DIRECTORY GOOGLE COLAB
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://www-us.apache.org/dist/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop3.2.tgz
#!tar -xvf spark-3.0.0-preview2-bin-hadoop3.2.tgz

!wget -q https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop3.2.tgz
!tar -xvf spark-3.0.2-bin-hadoop3.2.tgz

# INSATLL LIB. FINDSPARK UNTUK MENGIMPORT PYSPARK
!pip install -q findspark

spark-3.0.2-bin-hadoop3.2/
spark-3.0.2-bin-hadoop3.2/R/
spark-3.0.2-bin-hadoop3.2/R/lib/
spark-3.0.2-bin-hadoop3.2/R/lib/sparkr.zip
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/worker/
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/worker/worker.R
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/worker/daemon.R
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/tests/
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/profile/
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/profile/shell.R
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/profile/general.R
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/R/
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/R/SparkR
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/R/SparkR.rdx
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/R/SparkR.rdb
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/Meta/
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/Meta/features.rds
spark-3.0.2-bin-hadoop3.2/R/lib/SparkR/Meta/pac

In [2]:
# IMPORT LIB. OS UNTUK MEN-SETTING ENVIRONMENT GOOGLE COLAB
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop3.2"

# IMPORT BEBERAPA LIBRARY 

In [3]:
# INSTALL LIB. SASTRAWI UNTUK DIGUNAKAN PADA STEMMING
!pip install Sastrawi

     |████████████████████████████████| 215kB 12.6MB/s 


In [4]:
import pandas as pd   #--> LIB. PANDAS UNTUK MEMUAT DATAFRAME, TETAPI TIDAK DIGUNAKAN UNTUK PENGOLAHAN PADA ANALISIS SENTIMEN INI
import io as io       #--> LIB. IO UNTUK MEMBACA FILE INPUT YANG DIUPLAD KE DALAM GOOGLE COLAB
import requests as rq #--> LIB. REQUEST UNTUK MENGAMBIL STOPLIST TALA
import math           #--> LIB. MATH UNTUK MENGHITUNG LOG10 SAAT MENENTUKAN TERM FREQUENCY
import Sastrawi       #--> LIB. SASTRAWI UNTUK MEN-STEMMING TERM
import time           #--> LIB. TIME UNTUK MENGHITUNG RUNTIME EXCECUTION

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  #--> LIB. STEMMERFACTORY MERUPAKAN FUNCTION STEMMING 
from google.colab import files                              #--> LIB. FILES UNTUK MENGUPLOAD FILE DATASET KE DALAM GOOGLE COLAB

In [5]:
import findspark  #--> LIB. FINDSPARK UNTUK MENGAMBIL SPARK PADA DIRECTORY GOOGLE COLAB
findspark.init()  #--> FUNCTION MENGINISIASI SPARK PADA DIRECTORY GOOGLE COLAB
findspark.find()  #--> MENAMPILKAN LOKASI SPARK PADA DIRECTORY GOOGLE COLAB

'/content/spark-3.0.2-bin-hadoop3.2'

In [6]:
import pyspark  #--> LIB. PYSPARK SEBAGAI OPERASI RDD PADA ANALISIS SENTIMEN INI

from pyspark.sql import SparkSession                         #--> LIB. SPARKSESSION UNTUK MENJALANKAN PYSPARK
from pyspark.sql.functions import explode, concat, col, lit  #--> BEBERAPA LIB DARI PYSPARK UNTUK DIGUNAKAN

spark = SparkSession.builder.getOrCreate()  #--> MENJALANKAN PYSPARK
sc = spark.sparkContext

# MEMISAHKAN DATA LATIH & DATA UJI

## 1) MEMBUAT DATA LATIH

In [ ]:
def get_DF_DataLatih(dfNegatif, dfPositif):
  # MENGAMBIL DATA KELAS POSITIF & NEGATIF SECARA MERATA UNTUK DIJADIKAN DATA LATIH
  dfLatihPositif = spark.createDataFrame(dfPositif.take(int(drainData/2)))
  dfLatihNegatif = spark.createDataFrame(dfNegatif.take(int(drainData/2)))
  
  dfDataLatih = dfLatihNegatif.union(dfLatihPositif)  #--> MENGGABUNGKAN DATA KELAS POSITIF & NEGATIF YANG SUDAH DIBAGI RATA

  return dfDataLatih
  #dfDataLatih.show()

In [ ]:
def get_DF_DataLatih_KFOLD(dfDataset, dfDataUji):
  # MENGAMBIL DATA UJI DENGAN MEMBANDINGAKAN DATASET & DATA LATIH
  dfDataLatih = dfDataset.join(dfDataUji, ["Judul Berita", "Isi Berita", "Kelas Berita"], "leftanti") #--> MENGAMBIL DATA YANG BERBEDA DENGAN MENGGUNAKAN "LEFTANTI"
  dfDataLatih = dfDataLatih.sort("Kelas Berita", ascending=True)                                          #--> MENGURUTKAN DATA UJI BERDASARKAN NAMA KELAS 

  return dfDataLatih
  #dfDataLatih.show()

In [ ]:
def get_RDD_DataLatih(dfDataLatih):  
  df = dfDataLatih.select(col("Unnamed: 0"), col("Judul Berita"), col("Isi Berita")) #--> MEMILIH INDEKS BERITA, JUDUL BERITA, DAN ISI BERITA
  rddDataLatih = df.rdd.map(lambda x: ("D"+str(x[0]), str(x[1]+" "+x[2])))           #--> MENYIMPAN DATA LATIH KE DALAM SPARK RDD

  return rddDataLatih

## 2) MEMBUAT DATA UJI

In [ ]:
def get_DF_DataUji(dfDataset, dfDataLatih):
  # MENGAMBIL DATA UJI DENGAN MEMBANDINGAKAN DATASET & DATA LATIH
  dfDataUji = dfDataset.join(dfDataLatih, ["Judul Berita", "Isi Berita", "Kelas Berita"], "leftanti") #--> MENGAMBIL DATA YANG BERBEDA DENGAN MENGGUNAKAN "LEFTANTI"
  dfDataUji = dfDataUji.sort("Kelas Berita", ascending=True)                                          #--> MENGURUTKAN DATA UJI BERDASARKAN NAMA KELAS 

  return dfDataUji
  #dfDataUji.show()

In [ ]:
def get_DF_DataUji_KFOLD(dfNegatif, dfPositif):
  dfUjiNegatif = spark.createDataFrame(dfNegatif)
  dfUjiPositif = spark.createDataFrame(dfPositif)

  dfDataUji = dfUjiNegatif.union(dfUjiPositif)

  return dfDataUji
  #dfDataUji.show()

In [ ]:
def get_RDD_DataUji(dfDataUji):  
  df = dfDataUji.select(col("Unnamed: 0"), col("Judul Berita"), col("Isi Berita")) #--> MEMILIH INDEKS BERITA, JUDUL BERITA, DAN ISI BERITA
  rddDataUji = df.rdd.map(lambda x: ("D"+str(x[0]), str(x[1]+" "+x[2])))           #--> MENYIMPAN DATA UJI KE DALAM SPARK RDD

  return rddDataUji

# TEXT PREPROCESSING

##1) TOKENIZING

In [ ]:
# MEMBUAT FUNCTION TOKENIZE UNTUK MEMISAHKAN SETIAP KATA PADA DATA 
def Tokenize(rddValue):
  punc_Num='â¢Â€Ã!"#$%&\'()*+,:;<=>?@[\\]^_`{|}~0123456789'
  for ch in punc_Num:                           #--> PERULANGAN UNTUK MENGECEK CHARACTER BERDASARKAN "punc_Num"
    rddValue = rddValue.replace(ch, "")         #--> MENGHAPUS CHARACTER 
  rddValue = rddValue.replace(".", " ")         #--> MENGGANTIKAN CHARACTER "." MENJADI WHITESPACE " "
  rddValue = rddValue.replace("-", " ")         #--> MENGGANTIKAN CHARACTER "-" MENJADI WHITESPACE " "
  rddValue = rddValue.replace("/", " ")         #--> MENGGANTIKAN CHARACTER "/" MENJADI WHITESPACE " "
  rddValue = rddValue.replace("‚", " ")         #--> MENGGANTIKAN CHARACTER "," MENJADI WHITESPACE " "
  rddValue = rddValue.replace("\xa0", " ")      #--> MENGGANTIKAN CHARACTER "\xa0" MENJADI WHITESPACE " "
  rddValue = rddValue.replace("\x80\x9c", " ")  #--> MENGGANTIKAN CHARACTER "\x80\x9c" MENJADI WHITESPACE " "
  rddValue = rddValue.replace("\x80", " ")      #--> MENGGANTIKAN CHARACTER "\x80" MENJADI WHITESPACE " "
  rddValue = rddValue.replace("\x93", " ")      #--> MENGGANTIKAN CHARACTER "\x93" MENJADI WHITESPACE " "
  rddValue = rddValue.replace("\x94", " ")      #--> MENGGANTIKAN CHARACTER "\x94" MENJADI WHITESPACE " "
  rddValue = rddValue.replace("\x99", " ")      #--> MENGGANTIKAN CHARACTER "\x99" MENJADI WHITESPACE " "

  rddToken = rddValue.split()   #--> MEMISAHKAN SETIAP KATA
  
  return rddToken

In [ ]:
def Tokenizing(rddTextPrep):
  # PROSES MAPPING UNTUK MELAKUKAN TOKENIZING
  rddToken = rddTextPrep.map(lambda x: (x[0], Tokenize(x[1])))              #--> PROSES INISIALSASI (D+INDEKS) SEBAGAI KEY, TOKENIZING SEBAGAI VALUE)
  #rddToken = rddTextPrep.map(lambda x: ("D"+str(x[1]+1), Tokenize(x[0])))  #--> PROSES INISIALSASI (D+INDEKS) SEBAGAI KEY, TOKENIZING SEBAGAI VALUE)

  print("  Tokenizing Done. . .")
  return rddToken

##2) FILTERING

In [ ]:
def Filtering(rddToken):
  # MEMBUAT STOPLIST UNTUK FILTERING
  #stopList = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"] 
  stopList = []
  listBerita = ["cnn", "cnnindonesia", "cnnindonesiacom", "detik", "detikcom", "liputan", "liputancom", "inipasti", "inipasticom", "antaranews", "antaranewscom", "com"]

  # MENGAMBIL STOPLIST TALA
  url = "http://static.hikaruyuuki.com/wp-content/uploads/stopword_list_tala.txt" #--> LINK STOPLIST TALA
  headers = {'Accept-Encoding': 'identity'}
  r = rq.get(url, headers=headers); ind = 0;  #--> REQUEST LINK UNTUK MENGAMBIL STOPLIST TALA
  for line in r.text.splitlines():            #--> PERULANGAN UNTUK MENG-APPEND STOPLIST TALA KE DALAM "stopList"
    #print(line)
    stopList.append(line)
    ind += 1
    if len(r.text.splitlines()) == ind:     #--> MELAKUKAN PENAMBAHAN STOPLIST DARI "listBerita" APABILA SUDAH PADA PERULANGAN TERAKHIR
      for berita in listBerita:             #--> PERULANGAN UNTUK MENG-APPEND STOPLIST "listBerita" KE DALAM "stopList"
        stopList.append(berita)

  # PEOSES MAPPTING UNTK MELAKUKAN FILTERING
  rddFilter = rddToken.map(lambda x: (x[0], [item for item in x[1] if item.lower() not in stopList]))

  print("  Filtering Done. . .")
  return rddFilter

##3) STEMMING

In [ ]:
def Stemming(rddFilter):
  # MEMBUAT STEMLIST DARI FUNCTION STEMMERFACTORY
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()

  # PROSES MAPPING UNTUK MELAKUKAN STEMMING
  rddStem = rddFilter.map(lambda x: (x[0], [stemmer.stem(item) for item in x[1]]))

  print("  Stemming Done. . .")
  return rddStem

#TERM WEIGHTING

##1) MENGHITUNG TF

In [ ]:
def hitungTF(rddTerm):
  # MENGHITUNG TERM FREQUENCY DENGAN LOG-TF
  rddTF = rddTerm.flatMap(lambda x: [((x[0][0], item), 1) for item in x[1]])  #--> PROSES MAP MENGAMBIL VALUE DARI "rddTerm" DENGAN (DOK. INDEKS, TERM), 1 SEBAGAI VALUE
 
  rddTF = rddTF.reduceByKey(lambda x, y: x + y)                               #--> PROSES REDUCE MENJUMLAHKAN TOTAL VALUE SESUAI KEY YANG SAMA
  rddTF = rddTF.map(lambda x: (x[0], 1+math.log10(x[1])))                     #--> PROSES MAP (DOK. INDEKS, TERM) SEBAGAI KEY, PERHITUNGAN LOG-TF SEBAGAI VALUE

  print("  TF Done. . .")
  return rddTF

##2) MENGHITUNG DF

In [ ]:
def hitungDF(rddTerm):
  # ME-REDUCE KATA PADA MASING-MASING DOKUMEN
  rddDF = rddTerm.flatMap(lambda x: [((x[0][0], item), 1) for item in x[1]])   #--> PROSES MAP MENGAMBIL VALUE DARI "rddTerm" DENGAN (DOK. INDEKS, TERM), 1 SEBAGAI VALUE
  rddDF = rddDF.reduceByKey(lambda x, y: 1)                                    #--> PROSES REDUCE MERUBAH VALUE MENJADI 1 SESUAI KEY YANG SAMA PADA MASING-MASING DOK.

  # ME-REDUCE KATA PADA SELURUH DOKUMEN
  rddDF = rddDF.map(lambda x: (x[0][1], 1))         #--> PROSES MAP TERM SEBAGAI KEY, 1 SEBAGAI VALUE
  rddDF = rddDF.reduceByKey(lambda x, y: (x + y))   #--> PROSES REDUCE MENJUMLAHKAN VALUE SESUAI KEY YANG SAMA   

  print("  DF Done. . .")
  return rddDF

##3) MENGHITUNG IDF

In [ ]:
def hitungIDF(rddDF, countDoc):
  # MENGHITUNG IDF
  rddIDF = rddDF.map(lambda x: (x[0], math.log10(countDoc/x[1])))   #--> PROSES MAP TERM SEBAGAI KEY, PERHITUNGAN IDF SEBAGAI VALUE

  print("  IDF Done. . .")
  return rddIDF

##4) MENGHITUNG TF-IDF

In [ ]:
def hitungTFIDF(rddTF, rddIDF, rddKelasDataLatih):
  # MENYIAPKAN RDD UNTUK MENGHITUNG TF-IDF
  rddTFIDF = rddTF.map(lambda x: (x[0][1], (x[0][0], x[1])))          #--> PROSES MAP TERM SEBAGAI KEY, (DOC. INDEKS, TF) SEBAGAI VALUE
  rddTFIDF = rddTFIDF.join(rddIDF)                                    #--> PROSES MENGGABUNGKAN DENGAN "rddIDF"
  rddTFIDF = rddTFIDF.mapValues(lambda x: (x[0][0], x[0][1], x[1]))   #--> PROSES MAP TERM SEBAGAI KEY, (DOK. INDEKS, TF, IDF) SEBAGAI KEY

  # MENGHITUNG TF-IDF
  rddTFIDF = rddTFIDF.map(lambda x: (x[1][0], (x[0], x[1][1]*x[1][2]))) #--> PROSES MAP DOC. INDEKS SEBAGAI KEY, (TERM, TF-IDF) SEBAGAI VALUE

  # MENGAMBIL KELAS DARI SETIAP DATA LATIH
  rddTFIDF = rddTFIDF.join(rddKelasDataLatih)                                     #--> PROSES MENGGABUNGKAN DENGAN "rddKelasDataLatih", MENGAMBIl KELAS DARI DATA LATIH
  rddTFIDF = rddTFIDF.map(lambda x: ((x[0], x[1][0][0], x[1][1]), x[1][0][1]))    #--> PROSES MAP (DOC. INDEKS, TERM, KELAS) SEBAGAI KEY, TF-IDF SEBAGAI VALUE
  
  print("  TFIDF Done. . .")
  return rddTFIDF

#KLASIFIKASI

##1) MENGHITUNG PRIOR

In [ ]:
# rddKelasDataLatih = (D1, Positif), (D2, Positif), (D3, Negatif), (D4, Negatif)
# rddPrior = (Positif, 1), (Positif, 1), (Negatif, 1), (Negatif, 1)



def hitungPrior(rddKelasDataLatih, countDoc)
  # MENGAMBIL KELAS UNTUK MENGHITUNG PRIOR
  rddPrior = rddKelasDataLatih.map(lambda x: x[1])  #--> PROSES MAP UNTUK MENGAMBIL KELAS SEBAGAI VALUE

  # MENGHITUNG PRIOR
  rddPrior = rddPrior.map(lambda x: (x, 1))         #--> PROSES MAP KELAS SEBAGAI KEY, 1 SEBAGAI VALUE
  rddPrior = rddPrior.reduceByKey(lambda x, y: x + y).map(lambda x : (x[0], x[1]/countDoc))  #--> PRESES REDUCE MENGHITUNG VALUE DAN DI-MAP UNTUK MEMBAGI VALUE DENGAN TOTAL DATA LATIH

  print("  Prior Done. . .")
  return rddPrior


##2) MENGHITUNG LIKELIHOOD

In [ ]:
def hitungLikelihood(rddIDF, rddTFIDF, rddStem, rddDataUji, countTerm):       
  # MENGAMBIL NILAI IDF UNTUK MEMISAHKAN TERM MASING-MASING KELAS
  rddTermClass = rddIDF.map(lambda x: (x[0], ["Negatif", "Positif"]))              #--> PROSES MAP TERM SEBAGAI KEY, (NEGATIF, POISITF) SEBAGAI VALUE DARI "rddIDF"
  rddLikelihood = rddTermClass.flatMap(lambda x: [(item, x[0]) for item in x[1]])  #--> PROSES MAP MEMISAHKAN SETIAP KELAS, KELAS SEBAGAI KEY, TERM SEBAGAI VALUE

  # MENGHITUNG/COUNT SELURUH TW DARI MASING-MASING KELAS
  rddTotalTermClass = rddTFIDF.map(lambda x: (x[0][2], x[1]))             #--> PROSES MAP KELAS SEBAGAI KEY, TF-IDF SEBAGAI VALUE
  rddTotalTermClass = rddTotalTermClass.reduceByKey(lambda x, y: x + y)   #--> PROSES REDUCE MENJUMLAHKAN VALUE SESUAI KEY YANG SAMA

  # MENGGABUNGKAN HASIL COUNT SELURUH TW KE DALAM "rddLikelihood"
  rddLikelihood = rddLikelihood.join(rddTotalTermClass).map(lambda x: ((x[1][0], x[0]), x[1][1])) #--> MENGGABUNGKAN DENGAN "rddTotalTermClass" KEMUDIAN MELAKUKAN MAPPING (TERM, KELAS) SEBAGAI KEY, COUNT TW SEBAGAI VALUE
  
  # MENGHIUTNG/COUNT TERM TW DARI MASING-MASING KELAS
  rddTotalTermEachClass = rddTermClass.flatMap(lambda x: [((x[0], item), 0) for item in x[1]])    #--> PROSES MAP MEMISAHKAN SETIAP KELAS, (TERM, KELAS) SEBAGAI KEY, 0 SEBAGAI VALUE

  rddT_TFIDF_Term = rddTFIDF.map(lambda x: ((x[0][1], x[0][2]), x[1]))                                    #--> PROSES MAP (TERM, KELAS) SEBAGAI KEY, TF-IDF SEBAGAI VALUE DARI "rddTFIDF"
  rddT_TFIDF_Term = rddT_TFIDF_Term.reduceByKey(lambda x, y: x + y)                                       #--> PROSES REDUCE MENJUMLAHKAN VALUE SESUAI KEY YANG SAMA

  rddTotalTermEachClass = rddTotalTermEachClass.leftOuterJoin(rddT_TFIDF_Term)                            #--> MENGGABUNGKAN RDD UNTUK MENGAMBIL COUNT TERM TW DARI MASING-MASING KELAS
  rddTotalTermEachClass = rddTotalTermEachClass.mapValues(lambda x: x[1] if not x[1] is None else 0)      #--> MENGHITUNG COUNT TERM TW, APABILA VALUE "None" BERARTI COUNT TERM BERNILAI 0

  # MENGGABUNGKAN HASIL COUNT TERM TW KE DALAM "rddLikelihood"
  rddLikelihood = rddLikelihood.join(rddTotalTermEachClass).map(lambda x: (x[0][0], x[0][1], x[1][0], x[1][1]))    #--> MENGGABUNGKAN DENGAN "rddTotalTermEachClass" KEMUDIAN MELAKUKAN MAPPING (TERM, KELAS, COUNT TW, COUNT TERM TW) SEBAGAI VALUE
  ###rddLikelihood = rddLikelihood.reduceByKey(lambda k, v: k + v)                                                 #--> PROSES REDUCE MENJUMLAHKAN VALUE SESUAI KEY YANG SAMA

  # MELAKUKAN MAPPING "rddLikelihood" UNTUK MENGAMBIL NILAI TOTAL TERM
  rddLikelihood = rddLikelihood.map(lambda x: ("[KEY_COUNT_TERM]", x))    #--> PROSES MAP [KEY_COUNT_TERM] SEBAGAI KEY, (TERM, KELAS, COUNT TW, COUNT TERM TW) SEBAGAI VALUE
  rddLikelihood = countTerm.join(rddLikelihood)                           #--> MENGGABUNGKAN RDD UNTUK MENGAMBIL TOTAL TERM DATA LATIH DARI "countTerm"

  # MENGHITUNG MULTINOMIAL NAIVE BAYES (LIKELIHOOD)
  rddLikelihood = rddLikelihood.map(lambda x: (x[1][1][0], (x[1][1][1], (x[1][1][3] + 1)/(x[1][1][2] + x[1][0])))) 

  rddLikelihoodUji = hitungLikelihoodUji(rddStem, rddLikelihood, rddDataUji)

  print("  Likelihood Done. . .")
  return rddLikelihoodUji

In [ ]:
def hitungLikelihoodUji(rddStem, rddLikelihood, rddDataUji):
  # MEMPERSIAPKAN TERM DATA UJI
  rddStemDataUji = rddStem.join(rddDataUji)  #--> MENGAMBIL DATA UJI DENGAN MENGGABUNGKAN SESUAI KEY YANG ADA PADA "rddStem" & "rddStemDataUji"

  #PROSES MAPPING UNTUK MERUBAH KEY DAN VALUE "rddStemDataUji"
  rddStemDataUji = rddStemDataUji.map(lambda x: ("Q"+str(int(x[0][1:])), x[1][0]))  #--> QUERY INDEKS SEBAGAI KEY, TERM LIST SEBAGAI VALUE

  # MENJADIKAN TERM SEBAGAI DATA RDD
  rddTermUji = rddStemDataUji.flatMap(lambda x: [(item, x[0]) for item in x[1]])  #--> PROSES MAP TERM SEBAGAI KEY, QUERY INDEKS SEBAGAI VALUE

  # MEMPERSIAPKAN PERHITUNGAN LIKELIHOOD DATA UJI
  rddLikelihoodUji = rddTermUji.join(rddLikelihood).map(lambda x: ((x[1][0], x[1][1][0]), x[1][1][1])) #--> MENGGABUNGKAN "rddTermUji" DENGAN "rddLikelihood" KEMUDIAN MELAKUKAN MAPPING (QUERY INDEKS, KELAS) SEBAGAI KEY, LIKELIHOOD SEBAGAI VALUE (MEMBUANG TERM)

  # MENGHITUNG LIKELIHOOD DATA UJI
  decimalUP = 1000                                                                    #--> VALUE 100 UNTUK MENGANTISIPASI PANJANG DECIMAL SAAT MENGHITUNG LIKELIHOOD                          
  rddLikelihoodUji = rddLikelihoodUji.reduceByKey(lambda x, y: (x * y) * decimalUP)   #--> PROSES REDUCE MENGKALIKAN SELURUH NILAI LIKELIHOOD SESUAI KEY YANG SAMA
  #rddLikelihoodUji = rddLikelihoodUji.reduceByKey(lambda x, y: x * y)   #--> PROSES REDUCE MENGKALIKAN SELURUH NILAI LIKELIHOOD SESUAI KEY YANG SAMA
  rddLikelihoodUji = rddLikelihoodUji.map(lambda x: (x[0][1], (x[0][0], x[1])))       #--> PROSES MAP KELAS SEBAGAI KEY, (QUERY INDEKS, LIKELIHOOD) SEBAGAI VALUE

  return rddLikelihoodUji

##3) MENGHITUNG POSTERIOR

In [ ]:
def hitungPosterior(rddPrior, rddLikelihoodUji):
  # MENGGABUNGKAN "rddLikelihoodUji" DENGAN "rddPrior"
  rddPosterior = rddLikelihoodUji.join(rddPrior)
  rddPosterior = rddPosterior.map(lambda x: (x[1][0][0], (x[0], x[1][0][1] * x[1][1])))  #--> PROSES MAP QUERY INDEKS SEBAGAI KEY, (KELAS, HASIL PRIOR * LIKELIHOOD) SEBAGAI VALUE
  #rddPosterior = rddPosterior.sortByKey()  

  print("  Posterior Done. . .")
  return rddPosterior

# EKSEKUSI ANALISIS SENTIMEN

##1) MENGAMBIL DATASET

In [ ]:
# UPLOAD DATASET YANG DIINGINKAN (.XLSX)
uploaded = files.upload()
fileName = list(uploaded.keys())[0]

# MENGAMBIL HASIL UPLOAD DARI LIB. PANDAS, DAN DISIMPAN KE DALAM SPARK DATAFRAME  
df = pd.read_excel(io.BytesIO(uploaded[fileName]))     #--> "fileName" MERUPAKAN NAMA FILE YANG DIUPLOAD
dfDataset = spark.createDataFrame(df)                                           #--> MERUBAH PANDAS DATAFRAME MENJADI SPARK DATAFRAME

# MEMBUAT RDD DATASET
rddDataset = dfDataset.rdd                                                      #--> MEMBERIKAN INDEKS PADA MASING-MASING DATA
rddDataset = rddDataset.map(lambda x: ("D"+str(x[0]), (x[1], x[2], x[3])))      #--> PROSES MAP (D+INDEKS) SEBAGAI KEY, (JUDUL, ISI, KELAS) SEBAGAI VALUE

print("\n======================== DATASET ========================")
dfDataset.show()

Saving Dataset.xlsx to Dataset.xlsx

======================== DATASET ========================
+----------+--------------------+--------------------+------------+
|Unnamed: 0|        Judul Berita|          Isi Berita|Kelas Berita|
+----------+--------------------+--------------------+------------+
|         1|Usai Gelar Doa Be...|Jakarta, CNN Indo...|     Negatif|
|         2|Kubu Prabowo Klai...|Liputan6.com, Jak...|     Negatif|
|         3|Yenny Wahid: Joko...|Putri Presiden RI...|     Negatif|
|         4|Quick Count LSI D...|Berbagai lembaga ...|     Positif|
|         5|Prabowo - Sandi u...|Ambon (ANTARA) - ...|     Positif|
|         6|Disebut Bagian 1 ...|Liputan6.com, Jak...|     Negatif|
|         7|Prabowo: Saya Tak...|Liputan6.com, Jak...|     Negatif|
|         8|Pemantau Pemilu A...|Jakarta, CNN Indo...|     Positif|
|         9|Suara Jokowi-Ma'r...|Jakarta (ANTARA) ...|     Positif|
|        10|Relawan Jokowi Uc...|Liputan6.com, Jak...|     Negatif|
|        11|Prabowo S


##2) MAIN PROGRAM

In [ ]:
def save_RDD(rddSentimen, skenario, Supply=True):
  # MERUBAH MENJADI PANDAS, MEMPERBAIKI NAMA KOLOM
  dfSentimen = rddSentimen.map(lambda x: (x[0], x[1][0], x[1][1])).coalesce(2).toDF().toPandas()
  dfSentimen.columns = ["Dokumen", "Kelas", "Value"]
  dfSentimen.sort_values(by=["Dokumen"], inplace=True)
  dfSentimen.reset_index(drop=True, inplace=True)

  if Supply:
    fileName = "Skenario-1."+str(skenario+1)+".csv"
  else:
    fileName = "Skenario-2."+str(skenario+1)+".csv"
    
  dfFile = dfSentimen
  dfFile.to_csv(fileName, index=False)
  files.download(fileName)

In [ ]:
def printRDD(rddSpark, All=True):
  if All:
    print("  " + str(type(rddSpark)))
    print("  " + str(rddSpark.count()))
  for x in rddSpark.collect():
    print("  " + str(x))

###2.1) SUPPLY TRAINING TEST

In [ ]:
# MELAKUKAN PERHITUNGAN PERSENTASE ANTARA DATA LATIH & DATA UJI
persentase = [60, 70, 80, 90]
#persentase = [80]
totalDataset = dfDataset.count()                   #--> "totalDataset" MENGAMBIL TOTAL DATASET

for skenario, value in enumerate(persentase):
  start_time = time.time()                         #--> "startTime" MENGAMBIL CURRENT TIME
  drainData = round((totalDataset*value)/100)      #--> "drainData" SEBAGAI VARIABEL BERNILAI JUMLAH DATA LATIH YANG DIGUNAKAN

  print("============================================= SKENARIO - "+ str(skenario+1) + " =============================================")
  print("Jumlah Data Latih = " + str(drainData))
  print("Jumlah Data Uji   = " + str(totalDataset-drainData))
  
  print("\n> Memisahkan Data Latih & Data Uji")
  # MEMBUAT DATAFRAME BARU ANTARA POSITIF & NEGATIF
  dfPositif = dfDataset.filter(col("Kelas Berita") == "Positif")
  dfNegatif = dfDataset.filter(col("Kelas Berita") == "Negatif")

  dfDataLatih = get_DF_DataLatih(dfNegatif, dfPositif)
  rddDataLatih = get_RDD_DataLatih(dfDataLatih)
  dfDataUji = get_DF_DataUji(dfDataset, dfDataLatih)
  rddDataUji = get_RDD_DataUji(dfDataUji)
  print("  Done. . .")

  print("\n> Melakukan Text Preprocessing")
  # MENYIAPKAN RDD UNTUK PROSES TEXT PREPROCESSING
  rddTextPrep = rddDataset.map(lambda x: (x[0], str(x[1][0]+" "+x[1][1])))      #--> MENGAMBIL SELURUH DATA UNTUK DILAKUKAN TEKS PREPROCESSING
  
  # MENGGABUNGKAN RDD DATA LATIH DAN DATA UJI UNTUK DILAKUKAN TEXT PREPROCESSING
  #rddTextPrep = rddDataLatih.union(rddDataUji)
  #rddTextPrep = rddTextPrep.zipWithIndex()       #--> MEN-ZIP RDD UNTUK DIBERIKAN INDEKS PADA SETIAP DATA

  rddToken = Tokenizing(rddTextPrep)             #--> PROSES TOKENIZING
  rddFilter = Filtering(rddToken)                #--> PROSES FILTERING
  rddStem = Stemming(rddFilter)                  #--> PROSES STEMMING
  print("  Done. . .")

  print("\n> Melakukan Term Weighting")
  # MENGAMBIL KELAS PADA DATA LATIH
  dfKelasDataLatih = dfDataLatih.select(col("Kelas Berita"), col("Unnamed: 0")) #--> KELAS DATA LATIH DALAM BENTUK SPARK DATAFRAME
  rddKelasDataLatih = dfKelasDataLatih.rdd                                      #--> KELAS DATA LATIH DALAM BENTUK SPARK RDD
  
  # MEMBUAT KEY BERUPA INDEKS DOKUMEN
  rddKelasDataLatih = rddKelasDataLatih.map(lambda x: ("D"+str(x[1]), x[0]))    #--> PROSES MAP (D+INDEKS) SEBAGAI KEY, KELAS SEBAGAI VALUE

  # MEMBUAT TERM DATA LATIH
  rddTerm = rddKelasDataLatih.join(rddStem)                                     #--> MENGAMBIL DARI HASIL STEMMING DENGAN PROSES JOIN DARI "rddKelasDataLatih"
  rddTerm = rddTerm.map(lambda x: ((x[0], x[1][0]), x[1][1]))                   #--> PROSES MAP (DOK. INDEKS, KELAS) SEBAGAI KEY, STEM SEBAGAI VALUE

  countDoc = drainData                                                          #--> MENGAMBIL TOTAL DOKUMEN DATA LATIH
  #countDoc = rddTerm.count()                                                   

  rddTF = hitungTF(rddTerm)                                                     #--> HITUNG LOG-TF
  rddDF = hitungDF(rddTerm)                                                     #--> HITUNG DF
  rddIDF = hitungIDF(rddDF, countDoc)                                           #--> HITUNG IDF
  rddTFIDF = hitungTFIDF(rddTF, rddIDF, rddKelasDataLatih)                      #--> HITUNG TF-IDF
  print("  Done. . .")

  print("\n> Melakukan Analisis Sentimen")
  # MENGAMBIL TOTAL TERM DATA LATIH DISIMPAN KEDALAM RDD
  countTerm = rddIDF.map(lambda x: ("[KEY_COUNT_TERM]",1)).reduceByKey(lambda x, y: x + y)  #--> PROSES MAP [KEY_COUNT_TERM] SEBAGAI KEY, JUMLAH KATA SEBAGAI VALUE

  rddPrior = hitungPrior(rddKelasDataLatih, countDoc)                                       #--> HITUNG PRIOR
  rddLikelihood = hitungLikelihood(rddIDF, rddTFIDF, rddStem, rddDataUji, countTerm)        #--> HITUNG LIKELIHOOD
  rddPosterior = hitungPosterior(rddPrior, rddLikelihood)                                   #--> HITUNG POSTERIOR

  #printRDD(rddPosterior)
  print("  Done. . .")

  print("\n> Hasil Klasifikasi Masing-Masing Data Uji")
  # MENGAMBIL NILAI TERTINGGI (PELUANG TERTINGGI)
  rddSentimen = rddPosterior.reduceByKey(lambda x, y: (x[0], x[1]) if x[1] > y[1] else (y[0], y[1])) #--> PROSES REDUCE UNTUK MENCARI NILAI TERTINGGI DARI MASING-MASING KEY
  #printRDD(rddSentimen, False)
  print("  Done. . .")
  
  print("\n--- %.2f seconds ---" % (time.time() - start_time))
  #save_RDD(rddSentimen, skenario)
  #break

============================================= SKENARIO - 1 =============================================
Jumlah Data Latih = 60
Jumlah Data Uji   = 40

> Memisahkan Data Latih & Data Uji
  Done. . .

> Melakukan Text Preprocessing
  Tokenizing Done. . .
  Filtering Done. . .
  Stemming Done. . .
  Done. . .

> Melakukan Term Weighting
  TF Done. . .
  DF Done. . .
  IDF Done. . .
  TFIDF Done. . .
  Done. . .

> Melakukan Analisis Sentimen
  Prior Done. . .
  Likelihood Done. . .
  Posterior Done. . .
  Done. . .

> Hasil Klasifikasi Masing-Masing Data Uji
  Done. . .

--- 7.84 seconds ---
============================================= SKENARIO - 2 =============================================
Jumlah Data Latih = 70
Jumlah Data Uji   = 30

> Memisahkan Data Latih & Data Uji
  Done. . .

> Melakukan Text Preprocessing
  Tokenizing Done. . .
  Filtering Done. . .
  Stemming Done. . .
  Done. . .

> Melakukan Term Weighting
  TF Done. . .
  DF Done. . .
  IDF Done. . .
  TFIDF Done. . .
  

In [ ]:
printRDD(rddPosterior)

  <class 'pyspark.rdd.PipelinedRDD'>
  4
  ('Q8', ('Negatif', 1.0174248159797753e-173))
  ('Q7', ('Negatif', 8.112083706356538e-107))
  ('Q7', ('Positif', 1.0173410952115083e-112))
  ('Q8', ('Positif', 9.851889959821554e-169))


In [ ]:
Supply = True
  # MERUBAH MENJADI PANDAS, MEMPERBAIKI NAMA KOLOM
dfSentimen = rddSentimen.map(lambda x: (x[0], x[1][0], x[1][1])).coalesce(2).toDF().toPandas()
dfSentimen.columns = ["Dokumen", "Kelas", "Value"]
dfSentimen.sort_values(by=["Dokumen"], inplace=True)
dfSentimen.reset_index(drop=True, inplace=True)

if Supply:
  fileName = "Skenario-1."+str(skenario+1)+".csv"
else:
  fileName = "Skenario-2."+str(skenario+1)+".csv"
    
dfFile = dfSentimen
dfFile.to_csv(fileName, index=False)
files.download(fileName)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###2.2) K FOLD CROSS VALIDATION

In [ ]:
# MENENTUKAN JUMLAH K UNTUK PROSES FOLD
k = 2
divideData = (dfDataset.count()/2)/k
listDF_Negatif = []
listDF_Positif = []

dfNegatif = dfDataset.filter(col("Kelas Berita") == "Negatif").toPandas()
dfPositif = dfDataset.filter(col("Kelas Berita") == "Positif").toPandas()

for i in range(k):
  listDF_Negatif.append(dfNegatif.iloc[int(divideData*i):int(divideData*(i+1))])
  listDF_Positif.append(dfPositif.iloc[int(divideData*i):int(divideData*(i+1))])

for kFold in range(k):
  start_time = time.time()                                       #--> "startTime" MENGAMBIL CURRENT TIME
  drainData = dfDataset.count() - round(dfDataset.count()/k)     #--> "drainData" SEBAGAI VARIABEL BERNILAI JUMLAH DATA LATIH YANG DIGUNAKAN

  print("============================================= SKENARIO - "+ str(kFold+1) + " =============================================")
  print("Jumlah Data Latih = " + str(drainData))
  print("Jumlah Data Uji   = " + str(dfDataset.count()-drainData))
  
  print("\n> Memisahkan Data Latih & Data Uji")
  # MEMBUAT DATAFRAME BARU ANTARA POSITIF & NEGATIF
  dfDataUji = get_DF_DataUji_KFOLD(listDF_Negatif[kFold], listDF_Positif[kFold])
  rddDataUji = get_RDD_DataUji(dfDataUji)
  dfDataLatih = get_DF_DataLatih_KFOLD(dfDataset, dfDataUji)
  rddDataLatih = get_RDD_DataLatih(dfDataLatih)

  #dfDataUji = get_DF_DataUji(dfDataset, dfDataLatih)
  #rddDataUji = get_RDD_DataUji(dfDataUji)
  print("  Done. . .")

  print("\n> Melakukan Text Preprocessing")
  # MENYIAPKAN RDD UNTUK PROSES TEXT PREPROCESSING
  rddTextPrep = rddDataset.map(lambda x: (x[0], str(x[1][0]+" "+x[1][1])))      #--> MENGAMBIL SELURUH DATA UNTUK DILAKUKAN TEKS PREPROCESSING
  
  # MENGGABUNGKAN RDD DATA LATIH DAN DATA UJI UNTUK DILAKUKAN TEXT PREPROCESSING
  #rddTextPrep = rddDataLatih.union(rddDataUji)
  #rddTextPrep = rddTextPrep.zipWithIndex()       #--> MEN-ZIP RDD UNTUK DIBERIKAN INDEKS PADA SETIAP DATA

  rddToken = Tokenizing(rddTextPrep)             #--> PROSES TOKENIZING
  rddFilter = Filtering(rddToken)                #--> PROSES FILTERING
  rddStem = Stemming(rddFilter)                  #--> PROSES STEMMING
  print("  Done. . .")

  print("\n> Melakukan Term Weighting")
  # MENGAMBIL KELAS PADA DATA LATIH
  dfKelasDataLatih = dfDataLatih.select(col("Kelas Berita"), col("Unnamed: 0"))   #--> KELAS DATA LATIH DALAM BENTUK SPARK DATAFRAME
  rddKelasDataLatih = dfKelasDataLatih.rdd                                        #--> KELAS DATA LATIH DALAM BENTUK SPARK RDD
  
  # MEMBUAT KEY BERUPA INDEKS DOKUMEN
  rddKelasDataLatih = rddKelasDataLatih.map(lambda x: ("D"+str(x[1]), x[0]))      #--> PROSES MAP (D+INDEKS) SEBAGAI KEY, KELAS SEBAGAI VALUE

  # MEMBUAT TERM DATA LATIH
  rddTerm = rddKelasDataLatih.join(rddStem)                                     #--> MENGAMBIL DARI HASIL STEMMING DENGAN PROSES JOIN DARI "rddKelasDataLatih"
  rddTerm = rddTerm.map(lambda x: ((x[0], x[1][0]), x[1][1]))                   #--> PROSES MAP (DOK. INDEKS, KELAS) SEBAGAI KEY, STEM SEBAGAI VALUE

  countDoc = drainData                                                          #--> MENGAMBIL TOTAL DOKUMEN DATA LATIH
  #countDoc = rddTerm.count()                                                   

  rddTF = hitungTF(rddTerm)                                                     #--> HITUNG LOG-TF
  rddDF = hitungDF(rddTerm)                                                     #--> HITUNG DF
  rddIDF = hitungIDF(rddDF, countDoc)                                           #--> HITUNG IDF
  rddTFIDF = hitungTFIDF(rddTF, rddIDF, rddKelasDataLatih)                      #--> HITUNG TF-IDF
  print("  Done. . .")

  print("\n> Melakukan Analisis Sentimen")
  # MENGAMBIL TOTAL TERM DATA LATIH DISIMPAN KEDALAM RDD
  countTerm = rddIDF.map(lambda x: ("[KEY_COUNT_TERM]",1)).reduceByKey(lambda x, y: x + y)  #--> PROSES MAP [KEY_COUNT_TERM] SEBAGAI KEY, JUMLAH KATA SEBAGAI VALUE

  rddPrior = hitungPrior(rddKelasDataLatih, countDoc)                                       #--> HITUNG PRIOR
  rddLikelihood = hitungLikelihood(rddIDF, rddTFIDF, rddStem, rddDataUji, countTerm)        #--> HITUNG LIKELIHOOD
  rddPosterior = hitungPosterior(rddPrior, rddLikelihood)                                   #--> HITUNG POSTERIOR

  #printRDD(rddPosterior)
  print("  Done. . .")

  print("\n> Hasil Klasifikasi Masing-Masing Data Uji")
  # MENGAMBIL NILAI TERTINGGI (PELUANG TERTINGGI)
  rddSentimen = rddPosterior.reduceByKey(lambda x, y: (x[0], x[1]) if x[1] > y[1] else (y[0], y[1])) #--> PROSES REDUCE UNTUK MENCARI NILAI TERTINGGI DARI MASING-MASING KEY
  #printRDD(rddSentimen, False)
  print("  Done. . .")
  
  print("\n--- %.2f seconds ---" % (time.time() - start_time))
  #save_RDD(rddSentimen, kFold, False)
  #break

============================================= SKENARIO - 1 =============================================
Jumlah Data Latih = 50
Jumlah Data Uji   = 50

> Memisahkan Data Latih & Data Uji
  Done. . .

> Melakukan Text Preprocessing
  Tokenizing Done. . .
  Filtering Done. . .
  Stemming Done. . .
  Done. . .

> Melakukan Term Weighting
  TF Done. . .
  DF Done. . .
  IDF Done. . .
  TFIDF Done. . .
  Done. . .

> Melakukan Analisis Sentimen
  Prior Done. . .
  Likelihood Done. . .
  Posterior Done. . .
  Done. . .

> Hasil Klasifikasi Masing-Masing Data Uji
  Done. . .

--- 5.18 seconds ---
============================================= SKENARIO - 2 =============================================
Jumlah Data Latih = 50
Jumlah Data Uji   = 50

> Memisahkan Data Latih & Data Uji
  Done. . .

> Melakukan Text Preprocessing
  Tokenizing Done. . .
  Filtering Done. . .
  Stemming Done. . .
  Done. . .

> Melakukan Term Weighting
  TF Done. . .
  DF Done. . .
  IDF Done. . .
  TFIDF Done. . .
  

In [ ]:
printRDD(rddSentimen)

  <class 'pyspark.rdd.PipelinedRDD'>
  20
  ('Q13', ('Positif', 1.6806187227324713e-21))
  ('Q9', ('Positif', 956992590437.9546))
  ('Q4', ('Positif', 2059079681.2463188))
  ('Q8', ('Positif', 3.502637542626532e-25))
  ('Q5', ('Positif', 3132961876285533.5))
  ('Q17', ('Negatif', 1.836874065913085e-12))
  ('Q1', ('Positif', 2.6001650660938725e-10))
  ('Q3', ('Positif', 0.015042499942247442))
  ('Q10', ('Negatif', 0.7615935585376525))
  ('Q20', ('Positif', 159464.70440061396))
  ('Q14', ('Positif', 7.601443234325275e-11))
  ('Q6', ('Negatif', 1.5545720053244828e-09))
  ('Q7', ('Negatif', 1.0678737862918983e-06))
  ('Q16', ('Negatif', 3.383931280765734e-08))
  ('Q19', ('Positif', 545259057.8063266))
  ('Q11', ('Positif', 209803220830.41476))
  ('Q12', ('Positif', 3.838361458808537e-11))
  ('Q21', ('Negatif', 1.7668210477363984e-12))
  ('Q2', ('Positif', 1.4872640895076548e-05))
  ('Q15', ('Positif', 4.343689536069084e-22))


#DUMP

In [ ]:
def delokSek(rddSpark):
  for x in rddSpark.collect():
    #print(type(x))
    print(x)

In [ ]:
pandasDataset = dfDataset.toPandas()

display(pandasDataset)

In [ ]:
k = 5
divideData = (dfDataset.count()/2)/5
listDF_Negatif = []
listDF_Positif = []

dfNegatif = dfDataset.filter(col("Kelas Berita") == "Negatif").toPandas()
dfPositif = dfDataset.filter(col("Kelas Berita") == "Positif").toPandas()

for i in range(k):
  listDF_Negatif.append(dfNegatif.iloc[int(divideData*i):int(divideData*(i+1))])
  listDF_Positif.append(dfPositif.iloc[int(divideData*i):int(divideData*(i+1))])

In [ ]:
dfLatihNegatif = spark.createDataFrame(dfNegatif)
dfLatihPositif = spark.createDataFrame(dfPositif)

In [ ]:
dataset.filter(col("Kelas") == "Negatif").show()

In [ ]:
w = dfDataset.count()
print(w)

In [ ]:
# MENCOBA EXPLODE
rddTF = rddTerm.flatMap(lambda x: [((x[0][0], item), 1) for item in x[1]])
dfTF = rddTerm.toDF()

df_exploded = dfTF.withColumn('_2', explode('_2'))

dfTF.show()
df_exploded.show()

rddTF = df_exploded.rdd
rddTF = rddTF.map(lambda x: ((x[0][0], x[1]), 1))
printRDD(rddTF, False)

In [ ]:
start_time = time.time()

#countTerm = rddIDF                                              #--> MENGAMBIL TOTAL TERM DATA LATIH
#countTerm = countTerm.repartition(2).count()                                           #--> MENGAMBIL TOTAL TERM DATA LATIH

countTerm = rddIDF.countApprox(10,1.0)
print(countTerm)

print("--- %.2f seconds ---" % (time.time() - start_time))

In [ ]:

  rddStemDataLatih = rddTerm.map(lambda x: (x[0][0], x[1])) #--> MENGAMBIL "rddTerm" DENGAN PROSES MAP DOC. INDEKS SEBAGAI KEY, TERM SEBAGAI VALUE

  # MENGAMBIL DATA UJI DENGAN LEFT ANTI JOIN
  dfStemDataUji = rddStem.toDF().join(rddStemDataLatih.toDF(), ["_1", "_2"], "leftanti")  #--> MENGGABUNGKAN "rddStem" DENGAN "rddStemDataLatih" KEMUDIAN MENGABIL DATA YANG TIDAK SAMA
  rddStemDataUji = dfStemDataUji.select(col("_2")).rdd.flatMap(lambda x: list(x))         #--> MENGAMBIL KOLOM "_2" DARI DATAFRAME, KARENA BERISI TERM DATA UJI


  # PROSES MAPPING PADA DATA UJI
  rddStemDataUji = rddStemDataUji.zipWithIndex()                          #--> MEMBERIKAN INDEKS PADA DATA UJI
  rddStemDataUji = rddStemDataUji.map(lambda x: ("Q"+str(x[1]+1), x[0]))  #--> PROSES MAP (Q+INDEKS) SEBAGAI KEY, TERM LIST SEBAGAI VALUE
  #printRDD(rddStemDataUji, False) #<--


In [ ]:
def hitungLikelihood(countTerm, rddTFIDF, rddTerm, rddStem, rddDataUji):       
  # MENGAMBIL NILAI IDF UNTUK MEMISAHKAN TERM MASING-MASING KELAS
  rddLikelihood = rddIDF.map(lambda x: (x[0], ["Negatif", "Positif"]))              #--> PROSES MAP TERM SEBAGAI KEY, (NEGATIF, POISITF) SEBAGAI VALUE DARI "rddIDF"
  rddLikelihood = rddLikelihood.flatMap(lambda x: [(item, x[0]) for item in x[1]])  #--> PROSES MAP MEMISAHKAN SETIAP KELAS, KELAS SEBAGAI KEY, TERM SEBAGAI VALUE

  # MENGHITUNG/COUNT SELURUH TW DARI MASING-MASING KELAS
  rddTotalTermClass = rddTFIDF.map(lambda x: (x[0][2], x[1]))             #--> PROSES MAP KELAS SEBAGAI KEY, TF-IDF SEBAGAI VALUE
  rddTotalTermClass = rddTotalTermClass.reduceByKey(lambda x, y: x + y)   #--> PROSES REDUCE MENJUMLAHKAN VALUE SESUAI KEY YANG SAMA

  # MENGGABUNGKAN HASIL COUNT SELURUH TW KE DALAM "rddLikelihood"
  rddLikelihood = rddLikelihood.join(rddTotalTermClass).map(lambda x: ((x[1][0], x[0]), x[1][1])) #--> MENGGABUNGKAN DENGAN "rddTotalTermClass" KEMUDIAN MELAKUKAN MAPPING (TERM, KELAS) SEBAGAI KEY, COUNT TW SEBAGAI VALUE
  
  # MENGHIUTNG/COUNT TERM TW DARI MASING-MASING KELAS
  rddTotalTermEachClass = rddIDF.map(lambda x: (x[0], ["Negatif", "Positif"]))                            #--> PROSES MAP TERM SEBAGAI KEY, (NEGATIF, POISITF) SEBAGAI VALUE DARI "rddIDF"
  rddTotalTermEachClass = rddTotalTermEachClass.flatMap(lambda x: [((x[0], item), 0) for item in x[1]])   #--> PROSES MAP MEMISAHKAN SETIAP KELAS, (TERM, KELAS) SEBAGAI KEY, 0 SEBAGAI VALUE

  rddT_TFIDF_Term = rddTFIDF.map(lambda x: ((x[0][1], x[0][2]), x[1]))                                    #--> PROSES MAP (TERM, KELAS) SEBAGAI KEY, TF-IDF SEBAGAI VALUE DARI "rddTFIDF"
  rddT_TFIDF_Term = rddT_TFIDF_Term.reduceByKey(lambda x, y: x + y)                                       #--> PROSES REDUCE MENJUMLAHKAN VALUE SESUAI KEY YANG SAMA

  rddTotalTermEachClass = rddTotalTermEachClass.leftOuterJoin(rddT_TFIDF_Term)                              #--> MENGGABUNGKAN RDD UNTUK MENGAMBIL COUNT TERM TW DARI MASING-MASING KELAS
  rddTotalTermEachClass = rddTotalTermEachClass.mapValues(lambda x: x[0] + x[1] if not x[1] is None else 0) #--> MENGHITUNG COUNT TERM TW, APABILA VALUE "None" BERARTI COUNT TERM BERNILAI 0

  # MENGGABUNGKAN HASIL COUNT TERM TW KE DALAM "rddLikelihood"
  rddLikelihood = rddLikelihood.join(rddTotalTermEachClass).map(lambda x: (x[0][0], x[0][1], x[1][0], x[1][1]))    #--> MENGGABUNGKAN DENGAN "rddTotalTermEachClass" KEMUDIAN MELAKUKAN MAPPING (TERM, KELAS, COUNT TW, COUNT TERM TW) SEBAGAI VALUE
  ###rddLikelihood = rddLikelihood.reduceByKey(lambda k, v: k + v)                                                 #--> PROSES REDUCE MENJUMLAHKAN VALUE SESUAI KEY YANG SAMA

  # MELAKUKAN MAPPING "rddLikelihood" UNTUK MENGAMBIL NILAI TOTAL TERM
  rddLikelihood = rddLikelihood.map(lambda x: ("[KEY_COUNT_TERM]", x))    #--> PROSES MAP [KEY_COUNT_TERM] SEBAGAI KEY, (TERM, KELAS, COUNT TW, COUNT TERM TW) SEBAGAI VALUE
  rddLikelihood = countTerm.join(rddLikelihood)                           #--> MENGGABUNGKAN RDD UNTUK MENGAMBIL TOTAL TERM DATA LATIH DARI "countTerm"

  # MENGHITUNG MULTINOMIAL NAIVE BAYES (LIKELIHOOD)
  rddLikelihood = rddLikelihood.map(lambda x: (x[1][1][0], (x[1][1][1], (x[1][1][3] + 1)/(x[1][1][2] + x[1][0])))) 

  rddLikelihoodUji = hitungLikelihoodUji(rddTerm, rddStem, rddLikelihood, rddDataUji)

  print("  Likelihood Done. . .")
  return rddLikelihoodUji

In [ ]:
def get_RDD_DataLatih(dfDataLatih):  
  df = dfDataLatih.select(concat(col("Judul Berita"), lit(" "), col("Isi Berita"))) #--> MENGGABUNGKAN JUDUL BERITA DENGAN ISI BERITA
  rddDataLatih = df.rdd.flatMap(lambda x: list(x))                                  #--> MENYIMPAN DATA LATIH KE DALAM SPARK RDD

  return rddDataLatih

In [ ]:
def get_RDD_DataUji(dfDataUji):  
  df = dfDataUji.select(concat(col("Judul Berita"), lit(" "), col("Isi Berita"))) #--> MENGGABUNGKAN JUDUL BERITA DENGAN ISI BERITA
  rddDataUji = df.rdd.flatMap(lambda x: list(x))                                  #--> MENYIMPAN DATA LATIH KE DALAM SPARK RDD

  return rddDataUji

In [ ]:
def hitungLikelihoodUji(rddTerm, rddStem, rddLikelihood, rddDataUji):
  # MEMPERSIAPKAN TERM DATA UJI
  rddStemDataUji = rddStem.join(rddDataUji)  #--> MENGAMBIL DATA UJI DENGAN MENGGABUNGKAN SESUAI KEY YANG ADA PADA "rddStemn" & "rddStemDataUji"

  #PROSES MAPPING UNTUK MERUBAH KEY DAN VALUE "rddStemDataUji"
  rddStemDataUji = rddStemDataUji.map(lambda x: ("Q"+str(int(x[0][1:])), x[1][0]))  #--> QUERY INDEKS SEBAGAI KEY, TERM LIST SEBAGAI VALUE

  # MENJADIKAN TERM SEBAGAI DATA RDD
  rddTermUji = rddStemDataUji.flatMap(lambda x: [(item, x[0]) for item in x[1]])  #--> PROSES MAP TERM SEBAGAI KEY, QUERY INDEKS SEBAGAI VALUE

  # MEMPERSIAPKAN PERHITUNGAN LIKELIHOOD DATA UJI
  rddLikelihoodUji = rddTermUji.join(rddLikelihood).mapValues(lambda x: (x[0], x[1][0], x[1][1])) #--> MENGGABUNGKAN "rddTermUji" DENGAN "rddLikelihood" KEMUDIAN MELAKUKAN MAPPING (QUERY INDEKS, KELAS, LIKELIHOOD) SEBAGAI VALUE

  # MENGHITUNG LIKELIHOOD DATA UJI
  rddLikelihoodUji = rddLikelihoodUji.map(lambda x: ((x[1][0], x[1][1]), x[1][2]))                              #--> MEMBUANG TERM, PROSES MAP (QUERY INDEKS, KELAS) SEBAGAI KEY, LIKELIHOOD SEBAGAI VALUE
  rddLikelihoodUji = rddLikelihoodUji.reduceByKey(lambda x, y: x * y).map(lambda x: (x[0][1], (x[0][0], x[1]))) #--> PROSES REDUCE MENGKALIKAN SELURUH NILAI LIKELIHOOD, KEMUDIAN MELAKUKAN MAPPING KELAS SEBAGAI KEY, (QUERY INDEKS, LIKELIHOOD) SEBAGAI VALUE

  return rddLikelihoodUji